In [1]:
!pip install --upgrade ai2thor ai2thor-colab &> /dev/null
import ai2thor
import ai2thor_colab
import pickle
import os.path

from ai2thor.controller import Controller
from ai2thor_colab import (
    plot_frames,
    show_objects_table,
    side_by_side,
    overlay,
    show_video
)

ai2thor_colab.start_xserver()

"AI2-THOR Version: " + ai2thor.__version__

'AI2-THOR Version: 5.0.0'

In [23]:
#import ai2thor
#from ai2thor.controller import Controller
#controller = Controller(scene="FloorPlan10")

#controller = Controller(
#    agentMode="default",
#    visibilityDistance=1.5,
#    scene="FloorPlan209",
#
#    # step sizes
#    gridSize=0.25,
#    snapToGrid=True,
#    rotateStepDegrees=15,
#
#    # image modalities
#    renderDepthImage=True,
#    #renderInstanceSegmentation=False,
#    
#    # camera properties
#    width=600,
#    height=600,
#    fieldOfView=90
#)

controller = Controller(
    agentMode="default",
    visibilityDistance=1.5,
    scene="FloorPlan209",
    
    # image modalities
    #renderDepthImage=False,
    #renderInstanceSegmentation=False,
    
    # camera properties
    width=400,
    height=400,
    fieldOfView=120,
    
    # step sizes
    gridSize=0.25,
    snapToGrid=True,
    #rotateStepDegrees=15,
)

event = controller.step(action="RotateRight")
metadata = event.metadata
print(event, event.metadata.keys())
"AI2-THOR Version: " + ai2thor.__version__

<ai2thor.server.Event at 0x7f6092a0cee0
    .metadata["lastAction"] = RotateRight
    .metadata["lastActionSuccess"] = True
    .metadata["errorMessage"] = "
    .metadata["actionReturn"] = None
> dict_keys(['objects', 'isSceneAtRest', 'agent', 'heldObjectPose', 'arm', 'fov', 'cameraPosition', 'cameraOrthSize', 'thirdPartyCameras', 'collided', 'collidedObjects', 'inventoryObjects', 'sceneName', 'lastAction', 'errorMessage', 'errorCode', 'lastActionSuccess', 'screenWidth', 'screenHeight', 'agentId', 'depthFormat', 'colors', 'flatSurfacesOnGrid', 'distances', 'normals', 'isOpenableGrid', 'segmentedObjectIds', 'objectIdsInBox', 'actionIntReturn', 'actionFloatReturn', 'actionStringsReturn', 'actionFloatsReturn', 'actionVector3sReturn', 'visibleRange', 'currentTime', 'sceneBounds', 'actionReturn'])


'AI2-THOR Version: 5.0.0'

In [24]:
import time
#event = controller.step("MoveBack")
#plot_frames(controller.last_event)
frames = []
for _ in range(180 // 5):
    frames.append(controller.step(action="RotateLeft", degrees=5).frame)
    time.sleep(0.05)

In [4]:
#show_objects_table(controller.last_event.metadata['objects'])
print(controller.last_event.metadata['objects'])

[{'name': 'ArmChair_3e286e7a', 'position': {'x': -4.334003448486328, 'y': 0.001995176076889038, 'z': -3.5289852619171143}, 'rotation': {'x': -3.562221264491461e-12, 'y': 90.00016784667969, 'z': 2.4148366719600745e-06}, 'visible': False, 'isInteractable': False, 'receptacle': True, 'toggleable': False, 'isToggled': False, 'breakable': False, 'isBroken': False, 'canFillWithLiquid': False, 'isFilledWithLiquid': False, 'fillLiquid': None, 'dirtyable': False, 'isDirty': False, 'canBeUsedUp': False, 'isUsedUp': False, 'cookable': False, 'isCooked': False, 'temperature': 'RoomTemp', 'isHeatSource': False, 'isColdSource': False, 'sliceable': False, 'isSliced': False, 'openable': False, 'isOpen': False, 'openness': 0.0, 'pickupable': False, 'isPickedUp': False, 'moveable': True, 'mass': 28.299999237060547, 'salientMaterials': ['Fabric'], 'receptacleObjectIds': ['KeyChain|-04.21|+00.32|-03.65'], 'distance': 1.9607726335525513, 'objectType': 'ArmChair', 'objectId': 'ArmChair|-04.33|+00.00|-03.53'

In [34]:
objects = controller.last_event.metadata['objects']

for obj in objects:
    if obj['visible']:
        print(obj['objectType'] + " " + str(obj['visible']) + " " + str(obj['position']))

#from sklearn.utils import shuffle
#
#kitchens = ["FloorPlan" + str(i + 1) for i in range(30)]
#living_rooms = ["FloorPlan" + str(i + 201) for i in range(30)]
#bedrooms = ["FloorPlan" + str(i + 301) for i in range(30)]
#bathrooms = ["FloorPlan" + str(i + 401) for i in range(30)]
#
#floor_plans = kitchens + living_rooms + bedrooms + bathrooms
#labels = ["kitchen" for i in range(30)] + ["living room" for i in range(30)] + ["bedroom" for i in range(30)] + ["bathroom" for i in range(30)]
#
#floor_plans_shuffled,labels_shuffled = shuffle(floor_plans, labels, random_state=1983)
#
##print(kitchens)
##print(living_rooms)
##print(bedrooms)
##print(bathrooms)
##print(features)
##print(labels)
#print(floor_plans_shuffled)
#print(labels_shuffled)

ArmChair True {'x': -4.334003448486328, 'y': 0.001995176076889038, 'z': -3.5289852619171143}
Floor True {'x': -4.4715800285339355, 'y': -0.055619120597839355, 'z': -2.5462260246276855}
Newspaper True {'x': -4.237964630126953, 'y': 0.3441956341266632, 'z': -2.164184093475342}
Pillow True {'x': -4.282398700714111, 'y': 0.42073073983192444, 'z': -1.7359906435012817}
RemoteControl True {'x': -4.292991638183594, 'y': 0.33039507269859314, 'z': -2.663956880569458}
Sofa True {'x': -4.3549981117248535, 'y': 0.00991678237915039, 'z': -2.089966058731079}


In [35]:
#for i in range(len(floor_plans_shuffled)):
#for i in range(len(features_for_each_label), len(features_for_each_label) + 10):
#    #if i > 10: break
#    controller = Controller(
#        scene=floor_plans_shuffled[i]
#    )
#    #plot_frames(controller.last_event)    
#    objects = controller.last_event.metadata['objects']
#    objs_to_store = set() # we'll put our objects into a set to make sure that we don't repeat them more than once.
#    
#    for obj in objects:
#      #if not obj['visible']:
#      #print(obj['objectType']) 
#        objs_to_store.add(obj['objectType'])
#
#    # now we'll get the objects into a string separated by a space
#    objs_in_room_as_string = ""
#    for obj in objs_to_store:
#        objs_in_room_as_string += obj + " "
#        
#    #print(str(i + 1) + ") " + labels_shuffled[i] + " : " + str(objs_to_store))
#    print(str(i + 1) + ") " + labels_shuffled[i] + " : " + objs_in_room_as_string[:-1])
#    features_for_each_label.append(objs_in_room_as_string[:-1])
#   
## finally put what we've got into a pickle file
#print(features_for_each_label)


##controller = Controller()
##controller = Controller(
##    scene="FloorPlan210_physics"
##)
##plot_frames(controller.last_event)
##show_objects_table(controller.last_event.metadata['objects'])
##print(controller.last_event.metadata['objects'])
##objects = controller.last_event.metadata['objects']
##objs_to_store = set()
##for obj in objects:
##  #if not obj['visible']:
##  #print(obj['objectType']) 
##  objs_to_store.add(obj['objectType'])
##
##print(objs_to_store)